# Debuggers: Introduction to gdb, pdb

## Common gdb/pdb commands

### Both programs

- `print`, `p`: print the value of a variable
- `list`, `l`: list the code lines around the current line
- `up`: move a frame up the stack trace
- `down`: move a frame down the stack trace
- `backtrace`, `bt`: list all the frames in the stack (e.g. from main all the way down through all the subroutine calls)
- `next`, `n`: move to the next line of code, stepping over any subroutine calls
- `step`, `s`: similar to next, but will step into subroutine calls (even ones you might not expect like `vector::operator[]`, e.g. vector element access)
- `continue`, `c`: continue program execution until hitting the next breakpoint or until program execution finishes (whichever comes first)

### gdb

- `break`, `b`: tell the debugger to halt execution at a given point. Can be at the beginning of a method call, e.g. `break GroupDiffusion::computeQpResidual`, or a particular line of code, e.g. `break Kernel.C:50`
- `finish`: continue execution until the current frame or subroutine returns to the calling frame and print the returned value
- `return`: prematurely return from the current frame to the calling frame. Can optionally supply a value that you want the prematurely exiting frame to return
- `info <subcommand>`: get info about a particular subcommand. Most common:
- `info break`: get information about breakpoints
- `run`, `r`: run the program from the beginning
- `disable <breakpoint #>`: disable the supplied breakpoint
- `enable <breakpoint #>`: enable the supplied breakpoint
- `delete <breakpoint #>`, `d <breapoint #>`: delete the supplied breakpoint


### pdb

- `return`: like the `gdb finish` command; continue execution until the current method returns

## How I use gdb

`gdb --args <program to run>`

### Example: failed assert in C++ program

Some systems by default [don't break on failed assertions](http://stackoverflow.com/questions/33646862/what-is-the-proper-way-to-break-on-failed-asserts-in-gdb) (mine doesn't), meaning when the program fails, you can't inspect variables, etc. Solution is either

`break abort`

or 

`break exit`

After failing assertion, examine backtrace with `bt` and then move up the stack trace with `up` into user land code. Note that after typing a command once, subsequent `RET` without any input will execute that previous command. After a single `p xsec_names[j]`, I know exactly what the problem is.

### Example 2: inspect some routine

```
break CoupledFissionEigenKernel::computeQpResidual
break CoupledFissionEigenKernel::computeQpResidual if _qp == 1 && _group == 1
p _qp
p _group
l
n
n
p _chi[_qp][_group]
p computeConcentration((*_group_fluxes[i]), _qp)
```

Can try stepping many times to get into `computeConcentration`, but we'll discover all the `std::vector::operator[]` calls. Would be easier to just set a new breakpoint and then `continue`.


## How I use pdb

